In [1]:
'''
model: GRU
task: predict clip (15 way classifier)
data: all runs used together
input to model: clip time series/seq
output: label time series
'''
import numpy as np
import pandas as pd
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import KFold

from dataloader import _clip_class_df, _clip_class_rest_df, K_RUNS
from gru.dataloader import _get_clip_seq as _get_seq
from gru.models import GRUClassifier
from gru.cc_utils import _get_true_class_prob, _gru_acc, _gru_test_acc, _gruenc_test_traj
from sklearn.metrics import mean_squared_error, r2_score
from utils import _info
import argparse
import pickle
import time
import os
import statistics

import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
    tf.config.experimental.set_memory_growth(gpus[1], True)
    tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

# results directory
RES_DIR = 'results/clip_gru_recon'
if not os.path.exists(RES_DIR):
    os.makedirs(RES_DIR)

K_SEED = 330

In [2]:
class ARGS():
    input_data = 'data/roi_ts'
    roi = 300
    net = 7
    roi_name = 'roi'
    subnet = 'wb'
    
    zscore = 1
    
    k_fold = 10
    k_hidden = 32
    k_dim = 300
    k_layers = 1
    batch_size = 32
    num_epochs = 45
    train_size = 100
    
args = ARGS()

In [3]:
res_path = (RES_DIR + 
            '/%s_%d_net_%d' %(args.roi_name, args.roi, args.net) +
            '_trainsize_%d' %(args.train_size) +
            '_k_hidden_%d' %(args.k_hidden) +
            '_kdim_%d' %(args.k_dim) +
            '_k_layers_%d_batch_size_%d' %(args.k_layers, args.batch_size) +
            '_num_epochs_%d_z_%d.pkl' %(args.num_epochs, args.zscore))
        
mod_path = res_path.replace('results','models')
mod_path = mod_path.replace('pkl','h5')

gru_model_path = mod_path.replace('gru_recon', 'gru')
gru_model_path = gru_model_path.replace('_kdim_%d' %(args.k_dim), '')
args.gru_model_path = gru_model_path

# df = _clip_class_df(args)
with open('data/df.pkl', 'rb') as f:
    df = pickle.load(f)

In [4]:
# with open('data/df.pkl', 'wb') as f:
#     pickle.dump(df, f)

In [ ]:
'''
test subject results
view only for best cross-val parameters
'''
_info('test mode')
# get X-y from df
subject_list = df['Subject'].unique()
train_list = subject_list[:args.train_size]
test_list = subject_list[args.train_size:]

print('number of subjects = %d' %(len(subject_list)))
features = [ii for ii in df.columns if 'feat' in ii]
k_feat = len(features)
print('number of features = %d' %(k_feat))
args.k_class = len(np.unique(df['y']))
print('number of classes = %d' %(args.k_class))

# length of each clip
clip_time = np.zeros(args.k_class)
for ii in range(args.k_class):
    class_df = df[df['y']==ii]
    clip_time[ii] = np.max(np.unique(class_df['timepoint'])) + 1
clip_time = clip_time.astype(int) # df saves float
print('seq lengths = %s' %clip_time)

'''
init model
'''
# get train, test sequences
X_test, test_len, y_test = _get_seq(df, 
    test_list, args)
'''
train encoder
'''
accs = []
for perm in range(100):
    X_train, train_len, y_train = _get_seq(df, 
    train_list, args, label_shuffle=True)
    max_length = tf.math.reduce_max(train_len).numpy()


    model = GRUClassifier(X_train,
                          k_layers=args.k_layers,
                          k_hidden=args.k_hidden,
                          k_class = args.k_class)

    model.fit(X_train,y_train,epochs=args.num_epochs,
              validation_split=0.2,
              batch_size=args.batch_size,
              verbose=0)
    
    '''
    results on test data
    '''
    a, a_t, c_mtx = _gru_test_acc(model, X_test, y_test,
                                  clip_time, len(test_list))
    accs.append(np.mean(a))
    print('sacc = %0.6f' %statistics.mean(accs))

---
test mode
---
number of subjects = 176
number of features = 300
number of classes = 15
seq lengths = [ 84 245 222 189  65 227 260 250 181 186 205 143 233 231 256]
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
sacc = 0.087370
sacc = 0.087236
sacc = 0.084822
sacc = 0.083849
sacc = 0.083838
sacc = 0.083081
sacc = 0.084276
